# Library and Data Preparation 

In [1]:
import os
import fnmatch
import TWB.xliff as xliff
import spacy
import pandas as pd
import queue
import numpy as np
import unicodedata

In [2]:
# normalize unicode characters
def NFD(s):
        return unicodedata.normalize('NFD', s)

# put your data path here
data_dir = '/home/mixalis/Downloads/Translators Without Borders/'
metadata = pd.read_excel(data_dir + 'Hackathon-for-Good-2019_TWB-Challenge_Metadata.xlsx')

# choose only text documents
# TODO The metadata contain some files multiple times and the table is almost 3 times the number of documents
# Using various combinations of subsets of the columns to get all the 12156 documents did not succeed
accepted_documents = metadata.loc[(metadata['Format'] == 'doc') | (metadata['Format'] == 'pdf')].drop_duplicates()

# fix some problems with the encoding of special characters in filenames
accepted_documents['Filename'] = accepted_documents['Filename'].apply(NFD)

# update data path with the sdlxliff directory
data_dir += 'hackathon-for-good-2019_TWB-challenge_files/'

In [3]:
# store all sdlxliff filenames into a list
document_names = []

# trying to get all doc documents based on extensions
for document in os.listdir(data_dir):    
    if fnmatch.fnmatch(document,'*.sdlxliff'):
        if fnmatch.fnmatch(document,'*.doc*'):
            document_names.append(document)
        elif fnmatch.fnmatch(document,'*.DOC*'):
            document_names.append(document)
        elif fnmatch.fnmatch(document,'*.txt*'):
            document_names.append(document)    
        elif fnmatch.fnmatch(document,'*.pdf*'):
            document_names.append(document)
        elif fnmatch.fnmatch(document,'*.PDF*'):
            document_names.append(document)
        elif fnmatch.fnmatch(document,'*.odt*'):
            document_names.append(document)
        elif fnmatch.fnmatch(document,'*.rtf*'):
            document_names.append(document)
        elif fnmatch.fnmatch(document,'*.dotx*'):
            document_names.append(document)    

In [4]:
# get the translated contents of all documents
document_contents = []
document_source_langs = []
document_target_langs = []

for i in range(len(document_names)):
    document = document_names[i]
    temp_xliff = xliff.XLIFF(data_dir + document)
    document_contents.append(temp_xliff.target)
    document_source_langs.append(temp_xliff.source_lang)
    document_target_langs.append(temp_xliff.target_lang)

In [5]:
# same unicode fix as for accepted_documents
for i in range(len(document_names)):
    document_names[i] = NFD(document_names[i])

In [6]:
# remove the sdlxliff extension as most files in the metadata are without it
filenames = []
for i in range(len(document_names)):
    filenames.append(document_names[i].replace('.sdlxliff', ''))

In [7]:
# find the documents that most probably have no metadata, they are a lot
documents_without_metadata = []

count = 0
for name in filenames:
    temp = accepted_documents.loc[accepted_documents['Filename'] == name]
    if temp.empty:
        # for the special case that the translated version's extension is used
        temp2 = accepted_documents.loc[accepted_documents['Filename'] == name + '.sdlxliff']
        if temp2.empty:
            count += 1       
            documents_without_metadata.append(name)

print(count)

2733


In [8]:
# find the few entries in the metadata that are not in the documents 
# and the corresponding document filenames for the metadata filenames with wrong encoding
import difflib

filename_correspondence = {}
count = 0
for name in list(accepted_documents['Filename'].drop_duplicates()):
    if name not in filenames and name not in document_names:
        max_similarity = 0.0
        corresponding_document = None
        for filename in filenames:
            # filename letter similarity
            seq = difflib.SequenceMatcher(None, filename, name)
            if (seq.ratio() > max_similarity):
                max_similarity = seq.ratio()
                corresponding_document = filename
        # this threshold was manually checked and it produces only one false positive
        if max_similarity > 0.5:
            filename_correspondence[name] = corresponding_document
        else:
            count += 1
            print(name)

# delete the false positive 
del filename_correspondence['Patient_Release_Form_-_Final_4.25.docx']                
print(count)

Ø·Ø¹Ø§Ù_x0085_Ù_x0083_Ù_x0085__Ø§Ù_x0084_Ù_x008a_Ù_x0088_Ù_x0085_.docx
å_x0080__x008b_äººè_x0087_ªå_x0082_³.docx
2


In [9]:
# helper function that changes the wrongly encoded metadata filenames
def change_names(x):
    if x in filename_correspondence.keys():
        return filename_correspondence[x]
    else:
        return x

# start combining all present features/data per document
all_document_data = accepted_documents
all_document_data['Filename'] = accepted_documents['Filename'].apply(lambda x: change_names(x))

# combine document lines into one list instead of a list of lists for use in a DataFrame column
contents = []
for i in range(len(document_contents)):
    temp = ''
    for j in range(len(document_contents[i])):
        temp += document_contents[i][j] + ' '
    contents.append(temp)

# left inner join of DataFrames
# all_document_data contains information on all documents with content that have also metadata
name_contents = pd.DataFrame(columns=['Filename','Content'])
name_contents['Filename'] = filenames
name_contents['Content'] = contents
all_document_data = pd.merge(all_document_data,name_contents)

In [10]:
# verify which documents are missing from the previous DataFrame
s1 = set(all_document_data['Filename'])
s2 = set(name_contents['Filename'])
no_metadata = list(s2.difference(s1))

# create another DataFrame that contains only the filename, 
# source language, target language and content of these documents
no_metadata_source_langs = []
no_metadata_target_langs = []
no_metadata_contents = []

for i in no_metadata:
    j = filenames.index(i)
    no_metadata_source_langs.append(document_source_langs[j])
    no_metadata_target_langs.append(document_target_langs[j])
    no_metadata_contents.append(contents[j])

no_metadata_df = pd.DataFrame(columns=['Filename', 'Source_lang', 'Target_lang', 'Content'])
no_metadata_df['Source_lang'] = no_metadata_source_langs
no_metadata_df['Target_lang'] = no_metadata_target_langs
no_metadata_df['Content'] = no_metadata_contents
no_metadata_df['Filename'] = no_metadata